#### Notebook to feature engineer fraction of time PM10 was above EPA standard for all counties

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2019','2019_single_level_34_variables.nc')
sl_34_variables = xr.open_dataset(PATH_1) ## single level 34 variables
sl_34_variables
pm10=sl_34_variables['pm10'] 
pm10

<xarray.DataArray 'pm10' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2019-01-01 ... 2019-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/33)
    GRIB_paramId:                             210074
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_units:                               kg m**-3
    long_name:                                Particulate matter d <= 10 um
    units:                                    kg m**-3
    standard_name:                            mass_concentration_of_pm10_ambi...
    GRIB_number:                              0
    GRIB_surface:                             0.0

In [3]:
pm10.to_dataframe()

pm10
valid_time          latitude longitude                              
2019-01-01 00:00:00 49.25    -124.849999999999994  0.000000006402451
                             -124.099999999999994  0.000000007070898
                             -123.349999999999994  0.000000009600058
                             -122.599999999999994  0.000000008507453
                             -121.849999999999994  0.000000005264894
...                                                              ...
2019-12-31 21:00:00 24.50    -70.099999999999994   0.000000001093028
                             -69.349999999999994   0.000000002323041
                             -68.599999999999994   0.000000002854691
                             -67.849999999999994   0.000000002683095
                             -67.099999999999994   0.000000002250363

[7743840 rows x 1 columns]

In [4]:
pm_threshold=150*10**(-9) ## epa standard
pm_threshold

1.5000000000000002e-07

In [5]:
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2019_county_shapefile','cb_2019_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,48,081,01383826,0500000US48081,48081,Coke,06,2361153195,42331832,"POLYGON ((-100.82497 31.74941, -100.82415 31.8..."
1,48,273,01383922,0500000US48273,48273,Kleberg,06,2282572445,541041659,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ..."
2,48,203,01383887,0500000US48203,48203,Harrison,06,2331138836,40651525,"POLYGON ((-94.70215 32.45618, -94.70197 32.467..."
3,48,223,01383897,0500000US48223,48223,Hopkins,06,1987629163,65639829,"POLYGON ((-95.86333 33.04989, -95.86302 33.065..."
4,48,033,01383802,0500000US48033,48033,Borden,06,2324366073,22297606,"POLYGON ((-101.69128 32.96184, -101.55743 32.9..."
...,...,...,...,...,...,...,...,...,...,...
3228,20,119,00485024,0500000US20119,20119,Meade,06,2533222397,3258258,"POLYGON ((-100.65264 37.41669, -100.65264 37.4..."
3229,38,049,01035305,0500000US38049,38049,McHenry,06,4853487956,97323497,"POLYGON ((-101.05991 48.38598, -101.05988 48.4..."
3230,72,119,01804540,0500000US72119,72119,Río Grande,13,157008568,74876349,"POLYGON ((-65.87666 18.36314, -65.87598 18.364..."
3231,46,021,01266974,0500000US46021,46021,Campbell,06,1900214169,96243697,"POLYGON ((-100.49935 45.94363, -100.46284 45.9..."


In [6]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-100.82497 31.74941, -100.82415 31.8...
1       MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...
2       POLYGON ((-94.70215 32.45618, -94.70197 32.467...
3       POLYGON ((-95.86333 33.04989, -95.86302 33.065...
4       POLYGON ((-101.69128 32.96184, -101.55743 32.9...
                              ...                        
3228    POLYGON ((-100.65264 37.41669, -100.65264 37.4...
3229    POLYGON ((-101.05991 48.38598, -101.05988 48.4...
3230    POLYGON ((-65.87666 18.36314, -65.87598 18.364...
3231    POLYGON ((-100.49935 45.94363, -100.46284 45.9...
3232    POLYGON ((-96.45277 44.19689, -96.45267 44.254...
Name: geometry, Length: 3233, dtype: geometry

In [7]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0  -100.824968999999996  31.749413999999998
     1  -100.824151000000001  31.812061000000000
     2  -100.823909999999998  31.814214999999997
     3  -100.824312000000006  31.816344999999998
     4  -100.823738000000006  31.850076999999999
...                      ...                 ...
3232 89  -96.322918999999999  44.197381999999998
     90  -96.331584999999990  44.197348999999996
     91  -96.450322999999997  44.196978000000001
     92  -96.452774299617687  44.196779980888003
     93  -96.452773999999991  44.196894999999998

[1040851 rows x 2 columns]

#### Fraction of time PM10 was above EPA threshold

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time, PM2.5 was above EPA standard
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= pm10.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['pm10'].values[i] > pm_threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'PM10 above threshold':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3233

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 34min 25s
Wall time: 34min 54s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,PM10 above threshold
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3228,0.0
3229,0.0
3230,0.0
3231,0.0


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Coke,"POLYGON ((-100.82497 31.74941, -100.82415 31.8...",48081
1,Kleberg,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...",48273
2,Harrison,"POLYGON ((-94.70215 32.45618, -94.70197 32.467...",48203
3,Hopkins,"POLYGON ((-95.86333 33.04989, -95.86302 33.065...",48223
4,Borden,"POLYGON ((-101.69128 32.96184, -101.55743 32.9...",48033
...,...,...,...
3228,Meade,"POLYGON ((-100.65264 37.41669, -100.65264 37.4...",20119
3229,McHenry,"POLYGON ((-101.05991 48.38598, -101.05988 48.4...",38049
3230,Río Grande,"POLYGON ((-65.87666 18.36314, -65.87598 18.364...",72119
3231,Campbell,"POLYGON ((-100.49935 45.94363, -100.46284 45.9...",46021


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,PM10 above threshold
0,Coke,"POLYGON ((-100.82497 31.74941, -100.82415 31.8...",48081,0.0
1,Kleberg,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...",48273,0.0
2,Harrison,"POLYGON ((-94.70215 32.45618, -94.70197 32.467...",48203,0.0
3,Hopkins,"POLYGON ((-95.86333 33.04989, -95.86302 33.065...",48223,0.0
4,Borden,"POLYGON ((-101.69128 32.96184, -101.55743 32.9...",48033,0.0
...,...,...,...,...
3228,Meade,"POLYGON ((-100.65264 37.41669, -100.65264 37.4...",20119,0.0
3229,McHenry,"POLYGON ((-101.05991 48.38598, -101.05988 48.4...",38049,0.0
3230,Río Grande,"POLYGON ((-65.87666 18.36314, -65.87598 18.364...",72119,0.0
3231,Campbell,"POLYGON ((-100.49935 45.94363, -100.46284 45.9...",46021,0.0


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,PM10 above threshold
0,Coke,"POLYGON ((-100.82497 31.74941, -100.82415 31.8...",48081,0.0
1,Kleberg,"MULTIPOLYGON (((-97.31780 27.49456, -97.31590 ...",48273,0.0
2,Harrison,"POLYGON ((-94.70215 32.45618, -94.70197 32.467...",48203,0.0
3,Hopkins,"POLYGON ((-95.86333 33.04989, -95.86302 33.065...",48223,0.0
4,Borden,"POLYGON ((-101.69128 32.96184, -101.55743 32.9...",48033,0.0


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,PM10 above threshold
0,Coke,48081,0.000000000000000
1,Kleberg,48273,0.000000000000000
2,Harrison,48203,0.000000000000000
3,Hopkins,48223,0.000000000000000
4,Borden,48033,0.000000000000000
...,...,...,...
3228,Meade,20119,0.000000000000000
3229,McHenry,38049,0.000000000000000
3230,Río Grande,72119,0.000000000000000
3231,Campbell,46021,0.000000000000000


In [16]:
county_var.to_pickle('PM10_above_threshold.pkl')